In [150]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt


%matplotlib inline

In [151]:
def sample(d0, d1, n=32):
    x0 = d0.sample((n,))
    x1 = d1.sample((n,))
    y0 = torch.zeros((n, 1))
    y1 = torch.ones((n, 1))
    return torch.cat([x0, x1], 0), torch.cat([y0, y1], 0)

In [152]:
mu0 = torch.zeros(2)
mu1 = torch.Tensor([2, 2])
d0 = torch.distributions.MultivariateNormal(mu0, torch.eye(2))
d1 = torch.distributions.MultivariateNormal(mu1, torch.eye(2))
print(d0.sample((5,)))
print(d1.sample((5,)))

tensor([[ 1.4381,  1.6664],
        [ 1.7528, -1.7161],
        [ 0.1786,  0.2664],
        [ 1.2989, -0.1034],
        [-0.6019,  0.5499]])
tensor([[ 1.8386,  1.9996],
        [ 2.1965,  2.1737],
        [ 3.1720,  0.2938],
        [ 2.2794,  2.0033],
        [ 2.3801,  2.3457]])


In [154]:
#x = np.arange(-5, 5.1, 0.5)
#y = np.arange(-5, 5.1, 0.5)
#X,Y = np.meshgrid(x,y)
#XY=np.array([X.flatten(),Y.flatten()]).T
#XY

In [155]:
net = nn.Sequential(nn.Linear(2, 5), nn.ReLU(), nn.Linear(5, 1))
for p in net.parameters():
    print(p.data)
net_opt = optim.SGD(lr=1e-3, params=list(net.parameters()))

tensor([[-0.6404, -0.5861],
        [ 0.0322,  0.0330],
        [-0.5290, -0.4082],
        [-0.1702, -0.1710],
        [ 0.4290,  0.2475]])
tensor([-0.6991,  0.0616, -0.5209, -0.7068, -0.3391])
tensor([[-0.4027, -0.2975,  0.0133,  0.2140,  0.1143]])
tensor([ 0.3849])


In [156]:
log_freq = 500
for i in range(20000):
    if i%log_freq == 0:
        with torch.no_grad():
            x, y = sample(d0, d1, 100000)
            out = F.sigmoid(net(x))
            loss = F.binary_cross_entropy(out, y)
        print('Ошибка после %d итераций: %f' %(i/log_freq, loss))
    net_opt.zero_grad()
    x, y = sample(d0, d1, 1024)
    
#     out = F.sigmoid(layer(x))
#     loss = F.binary_cross_entropy(out, y)
    
    out = net(x)
    loss = F.binary_cross_entropy_with_logits(out, y)
    
    loss.backward()
    net_opt.step()

Ошибка после 0 итераций: 0.685465
Ошибка после 1 итераций: 0.651045
Ошибка после 2 итераций: 0.620990
Ошибка после 3 итераций: 0.588686
Ошибка после 4 итераций: 0.554204
Ошибка после 5 итераций: 0.521192
Ошибка после 6 итераций: 0.490879
Ошибка после 7 итераций: 0.462772
Ошибка после 8 итераций: 0.437812
Ошибка после 9 итераций: 0.415100
Ошибка после 10 итераций: 0.394758
Ошибка после 11 итераций: 0.375903
Ошибка после 12 итераций: 0.358645
Ошибка после 13 итераций: 0.343254
Ошибка после 14 итераций: 0.328127
Ошибка после 15 итераций: 0.316034
Ошибка после 16 итераций: 0.304095
Ошибка после 17 итераций: 0.294248
Ошибка после 18 итераций: 0.284307
Ошибка после 19 итераций: 0.275464
Ошибка после 20 итераций: 0.267525
Ошибка после 21 итераций: 0.261017
Ошибка после 22 итераций: 0.254597
Ошибка после 23 итераций: 0.250896
Ошибка после 24 итераций: 0.244657
Ошибка после 25 итераций: 0.238979
Ошибка после 26 итераций: 0.235400
Ошибка после 27 итераций: 0.233964
Ошибка после 28 итераций: 0.22

In [149]:
net

Sequential(
  (0): Linear(in_features=2, out_features=5, bias=True)
  (1): ReLU()
  (2): Linear(in_features=5, out_features=1, bias=True)
)